In [1]:
import scrapy
import requests
from scrapy.http import TextResponse

In [2]:
# 1. Start project

In [3]:
!scrapy startproject daangn

New Scrapy project 'daangn', using template directory '/Users/codefreak/opt/anaconda3/lib/python3.8/site-packages/scrapy/templates/project', created in:
    /Users/codefreak/code/02. python 강의/06_scrapy/daangn

You can start your first spider with:
    cd daangn
    scrapy genspider example example.com


In [10]:
# 3. items.py

In [11]:
# %load daangn/daangn/items.py

In [12]:
%%writefile daangn/daangn/items.py
import scrapy


class DaangnItem(scrapy.Item):
    category = scrapy.Field() 
    title = scrapy.Field() 
    price = scrapy.Field() 
    region = scrapy.Field() 
    user_id = scrapy.Field() 
    desc = scrapy.Field() 
    chat_counts = scrapy.Field() 
    watch_counts = scrapy.Field() 
    view_counts = scrapy.Field()

Overwriting daangn/daangn/items.py


In [13]:
# 4. spider.py

In [8]:
%%writefile daangn/daangn/spiders/spider.py
import scrapy
from daangn.items import DaangnItem

class DaangnSpider(scrapy.Spider):
    name = "Daangn"
    custom_settings = {
        'DOWNLOADER_MIDDLEWARES': {
            'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': None,
            'scrapy_fake_useragent.middleware.RandomUserAgentMiddleware': 400,
        }
    }

    
    
    def start_requests(self):
        url = "https://www.daangn.com/search/macbook"
        yield scrapy.Request(url, callback=self.parse)
        
    def parse(self, response):
        links = response.xpath('//*[@id="flea-market-wrap"]/article/a/@href').extract()
        links =  list(map(response.urljoin, links))
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):
        item = DaangnItem()
        item['category'] = response.xpath('//*[@id="article-category"]/text()')[0].extract().replace('∙', '').strip()
        item['title'] = response.xpath('//*[@id="article-title"]/text()')[0].extract().strip()
        item['price'] = response.xpath('//*[@id="article-price"]/text()')[0].extract().strip()
        desc = response.xpath('//*[@id="article-detail"]/p/text()').extract()
        item['desc'] = "".join(desc).replace('\n', '')
        item['user_id'] = response.xpath('//*[@id="nickname"]/text()')[0].extract().strip()
        item['region'] = response.xpath('//*[@id="region-name"]/text()')[0].extract().strip()
        counts = response.xpath('//*[@id="article-counts"]/text()')[0].extract().strip()
        item['chat_counts'] = counts.split('∙')[0].strip().split(' ')[1]
        item['watch_counts'] = counts.split('∙')[1].strip().split(' ')[1]
        item['view_counts'] = counts.split('∙')[2].strip().split(' ')[1]
       
        yield item

Overwriting daangn/daangn/spiders/spider.py


In [9]:
!tree daangn/

daangn/
├── daangn
│   ├── __init__.py
│   ├── __pycache__
│   │   ├── __init__.cpython-38.pyc
│   │   ├── items.cpython-38.pyc
│   │   └── settings.cpython-38.pyc
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       ├── __pycache__
│       │   ├── __init__.cpython-38.pyc
│       │   └── spider.cpython-38.pyc
│       └── spider.py
├── items.csv
└── scrapy.cfg

4 directories, 14 files


In [10]:
%%writefile run.sh
cd daangn
rm items.csv
scrapy crawl Daangn -o items.csv

Overwriting run.sh


In [11]:
!/bin/bash run.sh

2020-11-18 10:52:59 [scrapy.utils.log] INFO: Scrapy 2.4.0 started (bot: daangn)
2020-11-18 10:52:59 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.8.3 (default, Jul  2 2020, 11:26:31) - [Clang 10.0.0 ], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform macOS-10.15.7-x86_64-i386-64bit
2020-11-18 10:52:59 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-11-18 10:52:59 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'daangn',
 'NEWSPIDER_MODULE': 'daangn.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['daangn.spiders']}
2020-11-18 10:52:59 [scrapy.extensions.telnet] INFO: Telnet Password: b163bf67ebbb93a1
2020-11-18 10:52:59 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.fee

In [12]:
import pandas as pd

In [13]:
df = pd.read_csv("daangn/items.csv")

In [14]:
df

,category,chat_counts,desc,price,region,title,user_id,view_counts,watch_counts
0,디지털/가전,0,애플 apple new MacBook Retina 레티나 뉴맥북 Early 2015...,"500,000원",강서구 등촌제3동,"MacBook (Retina, Early 2015) 뉴맥북",S맨,274,6
1,디지털/가전,0,"2015년 맥북프로 13인치 판매 (MacBook Pro, Retina, 13-in...","550,000원",계양구 계산동,"2015년 맥북프로 13인치 판매 (MacBook Pro, Retina, 13-in...",DOMI,19,3
2,디지털/가전,9,상태 아주 좋습니다20년 3월 29일 구매했습니다Model No. A1932MacB...,"800,000원",제주시 이도2동,맥북 Macbook Air 256GB 8GB,아야,620,13
3,디지털/가전,0,[UAG 맥북 케이스] 맥북 프로( ***-****-**** ) 15인치 케이스가격...,"15,000원",동작구 사당동,[UAG 맥북 케이스] 맥북 프로(2016-2019) 15인치 케이스,김죄송c,36,0
4,디지털/가전,3,맥북 에어 13인치 2011mid i7 256gb 판매합니다.램은 4기가 입니다. ...,"240,000원",수원시 영통구 광교1동,가성비로 맥북 체험용 맥북 에어 13' 2011mid i7 256gb 판매합니다.,비수비수,838,18
5,디지털/가전,3,맥북프로 레티나 13인치 입니다액정은 기본 필름입니다.하단 배면부 생활기스 있습니다...,"500,000원",금천구 가산동,애플 맥북 프로 레티나 MacBook Pro A1502 13인치 판매,네오,517,6
